## Tokenization

In [ ]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    verdict = f.read()

import re
alltext = re.findall(r'\w+|[,.!?"\']|--',verdict)


alltext = sorted(set(alltext)) #sort the text

print(alltext)

vocab = {key:value for key,value in enumerate(alltext)} ## vocab
# for i,item in enumerate(vocab.items()):
#     if(i<50):
#         print(i,item)



In [ ]:
class SimpleClassTokenizer:
    def __init__(self,vocab):
        self.str_to_int = {s:i for i,s in vocab.items()}
        self.int_to_str = vocab
    
    def encode(self,text):
        preprocessed = re.findall(r'\w+|[,.!?"\']|--',text)
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
     
    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.!?"()\'])', r'\1', text)
        return text


In [ ]:
tokenizer = SimpleClassTokenizer(vocab)

text1 = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text1)
print(ids)
tokenizer.decode(ids)


In [ ]:
# print(alltext)
alltext2 = alltext
alltext2.extend(['<|endoftext|>','<|unk|>']) ## Add endline
print(len(alltext2))
vocab = {i:s for i,s in enumerate(alltext2)}
print(len(vocab))


In [ ]:
alltext.clear()

In [ ]:
class SimpleClassTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = {s:i for i,s in vocab.items()}
        self.int_to_str = vocab

    def encoding(self,text):
        preprocessed = re.findall(r'\w+|--|[,.:;\-()?!"\']',text)
        preprocessed = [item if item in self.str_to_int else '<|endoftext|>' for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decoding(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+(--|[,.:;\-()?!"\'])',r'\1',text)
        return text


In [ ]:
tokenizer = SimpleClassTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text0 = " <|endoftext|> ".join((text1, text2))

print(text0)
ids = tokenizer.encoding(text0)
print(ids)
print(tokenizer.decoding(ids))

## Byte Pair Encoding

In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
token_ids = tokenizer.encode(verdict,allowed_special={"<|endoftext|>"})
text = tokenizer.decode(token_ids)
print(len(token_ids))
print(len(text))
print(text[:50])

## Create Input and Output Pairs

In [ ]:
with open('the-verdict.txt',encoding='utf-8') as f:
    rawdata = f.read()
print(len(rawdata))

import tiktoken
import torch
from torch.utils.data import Dataset,DataLoader

class SimpleClassDataset(Dataset):
    def __init__(self,txt,tokenizer,stride, maxlength):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt,allowed_special={'<|endoftext|>'})

        for i in range(0,len(token_ids)-maxlength,stride):
            input_chunk = token_ids[i:i+maxlength]
            target_chunk = token_ids[i+1:i+maxlength+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index],self.target_ids[index]
    
def create_dataloader(text,stride=128,batch_size=4,max_length=256,shuffle=True,drop_last = True,num_worker=0):

    tokenizer = tiktoken.get_encoding('gpt2')
    
    dataset = SimpleClassDataset(text,tokenizer,stride,max_length)

    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_worker
    )
    
    return dataloader


dataloader = create_dataloader(text=rawdata,batch_size=4,stride=4,max_length=4,shuffle=False,drop_last=True)

data_iter = iter(dataloader)
input,target = next(data_iter)
print(input)
print(target)



20479
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922]])


## Token Embedding

In [ ]:
torch.manual_seed(123)
embedding_layer= torch.nn.Embedding(50257,256)
print(input.shape)
token_embedding = embedding_layer(input)
print(token_embedding.shape)

torch.Size([4, 4])
torch.Size([4, 4, 256])


## Positional Embedding

In [ ]:
max_length = 4
dimensions = 256
pos_embedding_layer = torch.nn.Embedding(max_length,dimensions)
pos_embedding = pos_embedding_layer(torch.arange(max_length))
print(pos_embedding)

tensor([[ 0.5423, -0.1224, -1.4150,  ...,  0.2515, -2.3067,  0.8155],
        [-0.3973, -1.2575, -1.9800,  ..., -0.1207,  0.3075, -0.6422],
        [ 0.1840,  1.1128,  1.0052,  ...,  0.2081,  0.5531, -1.1619],
        [ 1.4155,  0.6599,  0.3760,  ...,  0.7034, -0.6108,  0.1080]],
       grad_fn=<EmbeddingBackward0>)
